We want to log training sessions. 

There are two types of comments: instructions and comments. Instructions belong to the template and are not logged. They appear to the user and provide guidance on how to train.

Sessions can have comments, a date and units. Units are composed of an array of sets and comments. The smallest unit is therefore just a unit with a single set. This structure allows us to log the work performed in supersets, circuits, giant sets, etc. A lot of information (i.e. rest) is ommitted.

A set is a list of exercises. Exercises consist of a name, work and resistance. Work can be measured in reps or time (seconds, minutes, hours). Resistance is a list of resistances. A resistance consists of a unit and a value, such as 50 kilos, 15 tube, or 44 band. This structure omits calisthenic variations as resistance, but I might get to that later.

In [99]:
from dataclasses import dataclass, asdict

@dataclass
class Measurable:
    value: int
    unit: str

class Resistance(Measurable):
    pass

class Work(Measurable):
    pass
        
@dataclass
class Exercise:
    name: str
    work: Work
    resistances: list

In [100]:
Exercise('Pullover', Work(12,'reps'), Resistance(50, 'kilo'))

Exercise(name='Pullover', work=Work(value=12, unit='reps'), resistances=Resistance(value=50, unit='kilo'))

In [101]:
@dataclass
class Unit:
    name: str
    exercises: list
    comments: str = ""

from datetime import datetime

@dataclass
class Session:
    name: str
    date: datetime
    units: Unit
    comments: str = ""
    
    def to_dict(self):
        return asdict(self)


Now we want to parse from .wm (Workout Markdown) format to Session

In [102]:
workout = \
"""20200531 Lower 1
Lower 1
## The first line must be the title of the workout.
## Comments which start with ## will not be logged and can be used
## for instructions
# Comments that start with a single # will be logged. This comment
# will be logged under the session.
@ on mondays
## Use @ to write directives. The 'on' directive creates a new instance of the template file each week dated for the next monday.


++ A
## ++ Precedes the section name. In this case an EDT circuit.
## Can be performed at beginning or end of workout
## Perform as many sets in 15 minutes of:
# This is a comment of the unit.

+ Reverse flies
## A + precedes an exercise name
10 12b
## Results are logged directly after the name of an exercise.
12 12b
8 12b
+ Chest flies
12 10t 15t
## Resistance can be stacked
12 20t
12 20t 10t
+ Pulldown
12 30t
12 30t 10t
12 31t

++ B
## Pullup and pushup technique work
+ Pullup hangs
20s 80
## Time will be recognised automatically from number+{s,m,h}.
## Use weight in kilos to refer to bodyweight
+ Pushup planche
30s 80
+ Pullup holds on top
20s 80
+ Pushup bottom holds
20s 80
+ Pullup negatives
5 80

++ C
## Pullups and pushups
+ Pullups
3 80
2 80
+ Pushups
5 80
4 80

++ D
+ Triceps extension
+ Curls
"""

In [108]:
def parse_session(session_as_string):
    workout_lines = session_as_string.splitlines()
    session_name = workout_lines[1]
    workout_lines = workout_lines[2:]
    today = datetime.today()
    session_date = today.strftime("%Y%m%d")
    session_units = []
    session_comments = ""

    find_comments = True
    for i, line in enumerate(workout_lines):
        
        if not line or is_instruction(line):
            continue
            
        if is_comment(line) and find_comments:
            session_comments += process_comment(line)
            continue

        if is_start_of_unit(line):
            find_comments = False
            new_unit = parse_unit(workout_lines, i)
            session_units += [new_unit,]
    
    return Session(session_name, session_date, session_units, session_comments)

            
            
def parse_unit(workout_lines, i):
    unit_name = workout_lines[i].strip('++ ')
    unit_comments = ""
    unit_exercises = []
    find_comments = True
    workout_lines= workout_lines[i+1:]
    
    for j, line in enumerate(workout_lines):
        
        if not line or is_instruction(line):
            continue
            
        if is_comment(line) and find_comments:
            unit_comments += process_comment(line)
            continue
            
        if is_start_of_unit(line):
            break
    
        if is_start_of_exercise(line):
            find_comments = False
            new_exercises = parse_exercises(workout_lines, j)
            unit_exercises += new_exercises
        
        if is_start_of_unit(line):
            break
    
    return Unit(unit_name, unit_exercises, unit_comments)
            

def parse_exercises(workout_lines, j):
    exercise_name = workout_lines[j].strip('+ ')
    exercises = []
    for line in workout_lines[j+1:]:
        if not line or is_start_of_unit(line) or is_start_of_exercise(line):
            break
        if is_instruction(line):
            continue
        exercise = parse_exercise(exercise_name, line)
        exercises += [exercise,]
    return exercises


def parse_exercise(exercise_name, line):
    try:
        work, resistances = line.split(' ', maxsplit=1)
        work = parse_work(work)
        resistances = parse_resistances(resistances)
        return Exercise(exercise_name, work, resistances)
    except:
        print('Parsing failed')
        print(line)
    


import re
match_number = re.compile('[0-9]')

def parse_work(work_string):
    
    if match_number.match(work_string[-1]):
        return Work(int(work_string), 'repetitions')
    
    if work_string[-1]=='s':
        return Work(int(work_string[0:-1]), 'seconds')

    
def parse_resistances(resistances_string):
    resistances = []
    for resistance_string in resistances_string.split(' '):
        resistances += [parse_resistance(resistance_string)]
    return resistances

    
def parse_resistance(resistance_string):
    
    if resistance_string[-1]=='k':
        return Resistance(int(resistance_string[0:-1]), 'kilos')

    if resistance_string[-1]=='t':
        return Resistance(int(resistance_string[0:-1]), 'tube')
    
    if resistance_string[-1]=='b':
        return Resistance(int(resistance_string[0:-1]), 'band')
    
    
def is_instruction(line):
    if len(line)>=2:
        if line[0:2] == '##':
                return True
    return False


def is_comment(line):
    if len(line)>=1:
        if line[0]=='#':
            return True
    return False


def is_start_of_unit(line):
    if len(line)>=2:
        if line[0:2]=='++':
            return True
    return False


def is_start_of_exercise(line):
    if len(line)>=1:
        if line[0:1]=='+':
            return True
    return False


def process_comment(line):
    return line.strip('# ')+'\n'


parse_session(workout).to_dict()

{'name': 'Lower 1',
 'date': '20200531',
 'units': [{'name': 'A',
   'exercises': [{'name': 'Reverse flies',
     'work': {'value': 10, 'unit': 'repetitions'},
     'resistances': [{'value': 12, 'unit': 'band'}]},
    {'name': 'Reverse flies',
     'work': {'value': 12, 'unit': 'repetitions'},
     'resistances': [{'value': 12, 'unit': 'band'}]},
    {'name': 'Reverse flies',
     'work': {'value': 8, 'unit': 'repetitions'},
     'resistances': [{'value': 12, 'unit': 'band'}]},
    {'name': 'Chest flies',
     'work': {'value': 12, 'unit': 'repetitions'},
     'resistances': [{'value': 10, 'unit': 'tube'},
      {'value': 15, 'unit': 'tube'}]},
    {'name': 'Chest flies',
     'work': {'value': 12, 'unit': 'repetitions'},
     'resistances': [{'value': 20, 'unit': 'tube'}]},
    {'name': 'Chest flies',
     'work': {'value': 12, 'unit': 'repetitions'},
     'resistances': [{'value': 20, 'unit': 'tube'},
      {'value': 10, 'unit': 'tube'}]},
    {'name': 'Pulldown',
     'work': {'valu

In [55]:
session_date

datetime.datetime(2020, 5, 31, 12, 22, 14, 290974)